In [37]:
def ceiling_division(numer, denom):
    return -((-numer)//denom)

for i in range(1,1000):
    for j in range(1,1000):
        n = i // j
        if n > 3:
#             print(i,j,n)
            assert i // n >= j, (i,j,n,i//n)

In [32]:
24 // 4

6